# Robust Blind Face Restoration
- Source: https://github.com/sczhou/CodeFormer

In [ ]:
try:
    import torch
    import torchvision
except ImportError:
    !pip install torch
    !pip install torchvision

In [ ]:
if torch.cuda.is_available():
    print('Using GPU')
    device = 'cuda'
    !nvidia-smi
else:
    print('CUDA not available. Please connect to a GPU instance if possible.')
    device = 'cpu'

## Download your data

### Original video from YT

In [ ]:
import os
from os.path import exists as path_exists

if not path_exists('helper.py') and not 'CodeFormer' in os.getcwd():
    !wget https://raw.githubusercontent.com/machinelearnear/towards_robust_blind_face_restoration/main/helper.py

from helper import vid2frames, get_yt_video_id

youtube_url = 'https://www.youtube.com/watch?v=7K-Hk1qt_mk&t=0s&ab_channel=Entretenimientoyalgomas'
video_name = f'videos/{get_yt_video_id(youtube_url)}.mp4'

if not path_exists(video_name):
    skip_frames, path_frames = vid2frames(youtube_url, sample_all_frames=False, format_note='240p')

### Preview downloaded video

In [ ]:
from IPython.display import HTML
from base64 import b64encode

data_url = "data:video/mp4;base64," + b64encode(open(video_name, 'rb').read()).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

## Blind Face Restoration using [CodeFormer](https://github.com/sczhou/CodeFormer)

### Prepare environment

In [6]:
if not 'CodeFormer' in os.getcwd():
    %cd CodeFormer
    if not path_exists('CodeFormer'):
        !git clone https://github.com/sczhou/CodeFormer.git

try:
    import basicsr
    import facelib
except ImportError:
    # Install python dependencies
    !pip install -r requirements.txt

    # Install basicsr
    !python basicsr/setup.py develop

    # Download the pre-trained model
    !python scripts/download_pretrained_models.py facelib
    !python scripts/download_pretrained_models.py CodeFormer

'/home/studio-lab-user/towards_robust_blind_face_restoration'

### Run inference

In [2]:
if not 'CodeFormer' in os.getcwd():
    %cd CodeFormer
    
import os
import cv2
import argparse
import glob
import torch
from torchvision.transforms.functional import normalize
from basicsr.utils import imwrite, img2tensor, tensor2img
from basicsr.utils.download_util import load_file_from_url
from facelib.utils.face_restoration_helper import FaceRestoreHelper
import torch.nn.functional as F

from basicsr.utils.registry import ARCH_REGISTRY

from tqdm import tqdm

pretrain_model_url = {
    'restoration': 'https://github.com/sczhou/CodeFormer/releases/download/v0.1.0/codeformer.pth',
}

NameError: name 'os' is not defined

In [ ]:
class args:
    w = 0.7
    upscale = 2
    test_path = '../frames'
    has_aligned = False
    only_center_face = False
    draw_box = False
    bg_upsampler = 'realesrgan'
    bg_tile = 400

In [ ]:
# ------------------------ input & output ------------------------
if args.test_path.endswith('/'):  # solve when path ends with /
    args.test_path = args.test_path[:-1]

w = args.w
result_root = f'results/{os.path.basename(args.test_path)}_{w}'

In [ ]:
# ------------------ set up background upsampler ------------------
if args.bg_upsampler == 'realesrgan':
    if not torch.cuda.is_available():  # CPU
        import warnings
        warnings.warn('The unoptimized RealESRGAN is slow on CPU. We do not use it. '
                      'If you really want to use it, please modify the corresponding codes.',
                      category=RuntimeWarning)
        bg_upsampler = None
    else:
        from basicsr.archs.rrdbnet_arch import RRDBNet
        from basicsr.utils.realesrgan_utils import RealESRGANer
        model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=2)
        bg_upsampler = RealESRGANer(
            scale=2,
            model_path='https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.1/RealESRGAN_x2plus.pth',
            model=model,
            tile=args.bg_tile,
            tile_pad=10,
            pre_pad=0,
            half=True)  # need to set False in CPU mode
else:
    bg_upsampler = None

In [ ]:
# ------------------ set up CodeFormer restorer -------------------
net = ARCH_REGISTRY.get('CodeFormer')(dim_embd=512, codebook_size=1024, n_head=8, n_layers=9, 
                                        connect_list=['32', '64', '128', '256']).to(device)

# ckpt_path = 'weights/CodeFormer/codeformer.pth'
ckpt_path = load_file_from_url(url=pretrain_model_url['restoration'], 
                                model_dir='weights/CodeFormer', progress=True, file_name=None)
checkpoint = torch.load(ckpt_path)['params_ema']
net.load_state_dict(checkpoint)
net.eval()

In [ ]:
# ------------------ set up FaceRestoreHelper -------------------
# large det_model: 'YOLOv5l', 'retinaface_resnet50'
# small det_model: 'YOLOv5n', 'retinaface_mobile0.25'
face_helper = FaceRestoreHelper(
    args.upscale,
    face_size=512,
    crop_ratio=(1, 1),
    det_model = 'YOLOv5l',
    save_ext='png',
    use_parse=True,
    device=device)

In [1]:
# -------------------- start to processing ---------------------
# scan all the jpg and png images
for img_path in tqdm(sorted(glob.glob(os.path.join(args.test_path, '*.[jp][pn]g')))):
    # clean all the intermediate results to process the next image
    face_helper.clean_all()

    img_name = os.path.basename(img_path)
    # print(f'Processing: {img_name}')
    basename, ext = os.path.splitext(img_name)
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)

    if args.has_aligned: 
        # the input faces are already cropped and aligned
        img = cv2.resize(img, (512, 512), interpolation=cv2.INTER_LINEAR)
        face_helper.cropped_faces = [img]
    else:
        face_helper.read_image(img)
        # get face landmarks for each face
        num_det_faces = face_helper.get_face_landmarks_5(
            only_center_face=args.only_center_face, resize=640, eye_dist_threshold=5)
        # print(f'\tdetect {num_det_faces} faces')
        # align and warp each face
        face_helper.align_warp_face()

    # face restoration for each cropped face
    for idx, cropped_face in enumerate(face_helper.cropped_faces):
        # prepare data
        cropped_face_t = img2tensor(cropped_face / 255., bgr2rgb=True, float32=True)
        normalize(cropped_face_t, (0.5, 0.5, 0.5), (0.5, 0.5, 0.5), inplace=True)
        cropped_face_t = cropped_face_t.unsqueeze(0).to(device)

        try:
            with torch.no_grad():
                output = net(cropped_face_t, w=w, adain=True)[0]
                restored_face = tensor2img(output, rgb2bgr=True, min_max=(-1, 1))
            del output
            torch.cuda.empty_cache()
        except Exception as error:
            print(f'\tFailed inference for CodeFormer: {error}')
            restored_face = tensor2img(cropped_face_t, rgb2bgr=True, min_max=(-1, 1))

        restored_face = restored_face.astype('uint8')
        face_helper.add_restored_face(restored_face)

    # paste_back
    if not args.has_aligned:
        # upsample the background
        if bg_upsampler is not None:
            # Now only support RealESRGAN for upsampling background
            bg_img = bg_upsampler.enhance(img, outscale=args.upscale)[0]
        else:
            bg_img = None
        face_helper.get_inverse_affine(None)
        # paste each restored face to the input image
        restored_img = face_helper.paste_faces_to_input_image(upsample_img=bg_img, draw_box=args.draw_box)

    # save faces
    # for idx, (cropped_face, restored_face) in enumerate(zip(face_helper.cropped_faces, face_helper.restored_faces)):
    #     # save cropped face
    #     if not args.has_aligned: 
    #         save_crop_path = os.path.join(result_root, 'cropped_faces', f'{basename}_{idx:02d}.png')
    #         imwrite(cropped_face, save_crop_path)
    #     # save restored face
    #     if args.has_aligned:
    #         save_face_name = f'{basename}.png'
    #     else:
    #         save_face_name = f'{basename}_{idx:02d}.png'
    #     save_restore_path = os.path.join(result_root, 'restored_faces', save_face_name)
    #     imwrite(restored_face, save_restore_path)

    # save restored img
    if not args.has_aligned and restored_img is not None:
        save_restore_path = os.path.join(result_root, 'final_results', f'{basename}.png')
        imwrite(restored_img, save_restore_path)

print(f'\nAll results are saved in {result_root}')

NameError: name 'tqdm' is not defined

### Reconstruct restored video

In [ ]:
import cv2
import numpy as np
import glob
from pathlib import Path

img_array = []
for filename in glob.glob(f'{result_root}/final_results/*.png'):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width, height)
    img_array.append(img)

Path('../videos').mkdir(parents=True, exist_ok=True)
out = cv2.VideoWriter('../videos/restored.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 1, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
    
out.release()

In [ ]:
# https://www.arj.no/2021/05/11/ffmpeg-audio-video/
!ffmpeg -i ../$video_name -i ../videos/restored.mp4 -c copy -map 1:v:0 -map 0:a:0 -shortest ../videos/restored_w_audio.mp4

In [ ]:
data_url = "data:video/mp4;base64," + b64encode(open('../videos/restored_w_audio.mp4', 'rb').read()).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)